In [1]:
import sys

sys.path.append("..")

In [2]:
import os
from aurelio_sdk import AsyncAurelioClient

api_key = os.environ["AURELIO_API_KEY"]

client = AsyncAurelioClient(api_key=api_key)

print(client.api_key)
print(client.base_url)


sc_aT77r3LVSF7eUKrX3ev7feg.vvGE.acc-k79p1lhkkcpdd.MC4CAQAwBQYDK2VwBCIEICKG0/wTZM6F5QRQqjPCQRZ3fAORCDY9Asw0rfNXK+Az
https://api.aurelio.ai


## Chunk documents

Load data for chunking

In [3]:
from datasets import load_dataset

data = load_dataset("jamescalam/ai-arxiv2", split="train")
data

/Users/jakit/customers/aurelio/aurelio-sdk/.venv_312/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Dataset({
    features: ['id', 'title', 'summary', 'source', 'authors', 'categories', 'comment', 'journal_ref', 'primary_category', 'published', 'updated', 'content', 'references'],
    num_rows: 2673
})

In [4]:
content: str = data[3]["content"]
print(content[:1000])

# Mamba: Linear-Time Sequence Modeling with Selective State Spaces
# Albert Gu*1 and Tri Dao*2
1Machine Learning Department, Carnegie Mellon University 2Department of Computer Science, Princeton University agu@cs.cmu.edu, tri@tridao.me
# Abstract
Foundation models, now powering most of the exciting applications in deep learning, are almost universally based on the Transformer architecture and its core attention module. Many subquadratic-time architectures such as linear attention, gated convolution and recurrent models, and structured state space models (SSMs) have been developed to address Transformersâ computational ineï¬ciency on long sequences, but they have not performed as well as attention on important modalities such as language. We identify that a key weakness of such models is their inability to perform content-based reasoning, and make several improvements. First, simply letting the SSM parameters be functions of the input addresses their weakness with discrete modalities

### Regex chunker

In [5]:
from aurelio_sdk import ChunkingOptions, ChunkResponse

# All options are optional
chunking_options = ChunkingOptions(
    chunker_type="regex", delimiters=[], max_chunk_length=400
)

response_regex: ChunkResponse = await client.chunk(
    content=content, processing_options=chunking_options
)

In [ ]:
import json

for chunk in response_regex.document.chunks[:3]:
    print(json.dumps(chunk.model_dump(), indent=2))
    print("-" * 100)

### Semantic chunker

In [6]:
from aurelio_sdk import ChunkingOptions, ChunkResponse

# All options are optional
chunking_options = ChunkingOptions(
    chunker_type="semantic", max_chunk_length=400, window_size=5
)

response_semantic: ChunkResponse = await client.chunk(
    content=content, processing_options=chunking_options
)

In [ ]:
import json

for chunk in response_semantic.document.chunks[:3]:
    print(json.dumps(chunk.model_dump(), indent=2))
    print("-" * 100)

#### Compare `regex` and `semantic` chunks
Helper function to compare chunks

In [ ]:
from IPython.display import HTML


def display_chunks_side_by_side(semantic_chunks, regex_chunks):
    """Display concatenated texts with chunks in different background colors side by side on white background"""
    print(f"Semantic chunks: {len(semantic_chunks)}")
    print(f"Regex chunks: {len(regex_chunks)}")

    colors = ["#FFC0CB", "#ADD8E6", "#90EE90", "#FFFFE0", "#D3D3D3"]

    semantic_html = ""
    for idx, chunk in enumerate(semantic_chunks):
        color = colors[idx % len(colors)]
        semantic_html += (
            f'<span style="background-color:{color};">{chunk.content}</span>'
        )

    regex_html = ""
    for idx, chunk in enumerate(regex_chunks):
        color = colors[idx % len(colors)]
        regex_html += f'<span style="background-color:{color};">{chunk.content}</span>'

    html = f"""
    <table style="width:100%; table-layout:fixed; background-color:white;">
        <tr>
            <th style="width:50%; text-align:left;">Semantic Chunking</th>
            <th style="width:50%; text-align:left;">Regex Chunking</th>
        </tr>
        <tr>
            <td style="vertical-align: top; text-align:left;">{semantic_html}</td>
            <td style="vertical-align: top; text-align:left;">{regex_html}</td>
        </tr>
    </table>
    """

    display(HTML(html))

In [ ]:
display_chunks_side_by_side(
    response_semantic.document.chunks, response_regex.document.chunks
)

In [ ]:
display_chunks_side_by_side(
    response_semantic.document.chunks, response_regex.document.chunks
)

In [8]:
import aiofiles

async def read_file_stream(filepath: str) -> AsyncIterator[bytes]:
    """Read file as async stream of bytes"""
    async with aiofiles.open(filepath, mode='rb') as file:
        while chunk := await file.read(8192):  # 8KB chunks
            yield chunk


read_stream = read_file_stream("data/pdf/adaptive_semantic_search.pdf")

await upload_stream(read_stream)


ClientConnectorSSLError: Cannot connect to host localhost:8000 ssl:default [[SSL] record layer failure (_ssl.c:1000)]